In [23]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

view_log_train = pd.read_csv('../data/view_log.csv')
article_info = pd.read_csv('../data/article_info.csv')
submission = pd.read_csv('../submission/sample_submission.csv')

In [24]:
article_info.head()

,articleID,Title,Content,Format,Language,userID,userCountry,userRegion
0,ARTICLE_0000,19 Tips For Everyday Git Use,I've been using git full time for the past 4 y...,HTML,en,USER_0683,NaN,NaN
1,ARTICLE_0001,Intel buys computer vision startup Itseez to i...,Intel has acquired computer vision and machine...,HTML,en,USER_1129,NaN,NaN
2,ARTICLE_0002,Practical End-to-End Testing with Protractor,One of the reasons AngularJS is so great to wo...,HTML,en,USER_0256,NaN,NaN
3,ARTICLE_0003,Corporate venture growth in Brazil is another ...,Despite recent positive news and a renewed int...,HTML,en,USER_1304,NaN,NaN
4,ARTICLE_0004,Cross-channel user experiences with Drupal (aw...,"Last year around this time, I wrote that The B...",HTML,en,USER_0336,NaN,NaN


In [32]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

In [33]:
# 사전학습된 모델과 토크나이저 로드
model_name = 'nlptown/bert-base-multilingual-uncased-sentiment'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

C:\Users\SJ\anaconda3\envs\study\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\SJ\.cache\huggingface\hub\models--nlptown--bert-base-multilingual-uncased-sentiment. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [34]:
# GPU 사용 여부 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [35]:
# 예측 함수 정의
def predict(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    return predictions.item()

In [43]:
predicted_labels = []

for text in article_info['Content']:
    label = predict(text)
    predicted_labels.append(label)

In [44]:
predicted_labels

[3,
 3,
 3,
 3,
 1,
 3,
 3,
 3,
 3,
 3,
 4,
 3,
 3,
 1,
 1,
 4,
 4,
 4,
 0,
 2,
 4,
 3,
 1,
 1,
 3,
 0,
 4,
 4,
 1,
 1,
 1,
 2,
 3,
 2,
 1,
 3,
 3,
 3,
 3,
 3,
 1,
 3,
 1,
 3,
 3,
 3,
 3,
 3,
 1,
 3,
 1,
 4,
 4,
 3,
 4,
 0,
 3,
 3,
 3,
 2,
 1,
 1,
 3,
 1,
 4,
 0,
 2,
 4,
 1,
 4,
 0,
 3,
 3,
 3,
 3,
 3,
 1,
 3,
 3,
 3,
 0,
 1,
 2,
 1,
 0,
 3,
 3,
 4,
 1,
 4,
 1,
 0,
 1,
 2,
 4,
 1,
 3,
 4,
 3,
 3,
 3,
 0,
 1,
 1,
 3,
 0,
 1,
 1,
 1,
 1,
 3,
 0,
 3,
 1,
 3,
 1,
 3,
 3,
 3,
 3,
 1,
 3,
 4,
 3,
 2,
 4,
 4,
 1,
 3,
 3,
 3,
 3,
 4,
 4,
 3,
 3,
 3,
 2,
 1,
 3,
 3,
 3,
 1,
 1,
 3,
 2,
 1,
 1,
 1,
 3,
 3,
 1,
 0,
 4,
 0,
 4,
 1,
 3,
 2,
 3,
 3,
 0,
 1,
 3,
 4,
 3,
 1,
 1,
 3,
 0,
 3,
 4,
 3,
 3,
 3,
 1,
 1,
 1,
 3,
 3,
 3,
 3,
 1,
 3,
 3,
 1,
 3,
 4,
 1,
 1,
 1,
 1,
 3,
 1,
 3,
 3,
 3,
 3,
 3,
 1,
 1,
 0,
 3,
 1,
 0,
 3,
 4,
 1,
 3,
 4,
 0,
 1,
 1,
 4,
 0,
 3,
 3,
 4,
 4,
 3,
 1,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 0,
 3,
 3,
 1,
 3,
 1,
 3,
 0,
 3,
 0,
 0,
 3,
 4,
 2,
 4,
 3,
 2,
 0,
 2,
 2,
 0,
 1,
